In [1]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime

from random import randint
from numpy import array
from numpy import argmax
from random import randint

from math import sin
from math import pi
from math import exp
from random import randint
from random import uniform

from numpy import array
from math import ceil
from math import log10

from random import random
from numpy import cumsum
from numpy import array_equal

from sklearn.preprocessing import MinMaxScaler

import re

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

In [2]:
path = './Datasets/'

In [3]:
log_df = pd.read_parquet(path + "logs.parquet")

In [4]:
log_df.head(10)

,resp_time,bytes_sent,resp_code,url,datetime,app_1_containers,app_2_containers,app_3_containers,app_4_containers
0,0.031,197,200,/2/4-3/65,2019-09-05 22:37:10,3,3,3,3
1,0.380,198,200,/3/3-4/37,2019-09-05 22:37:10,3,3,3,3
2,0.298,188,200,/4/3/188,2019-09-05 22:37:11,3,3,3,3
3,0.009,188,200,/3/3/180,2019-09-05 22:37:11,3,3,3,3
4,0.331,199,200,/3/3-4/115,2019-09-05 22:37:11,3,3,3,3
5,0.148,229,200,/3/3-3-3-3-3/101,2019-09-05 22:37:11,3,3,3,3
6,0.023,188,200,/3/3/180,2019-09-05 22:37:11,3,3,3,3
7,0.298,189,200,/4/3/188,2019-09-05 22:37:11,3,3,3,3
8,0.027,207,200,/3/4-2-2/38,2019-09-05 22:37:11,3,3,3,3
9,0.352,210,200,/3/4-4-3/60,2019-09-05 22:37:11,3,3,3,3


In [5]:
# IMPORTANT!! - deleting all logs without a response time (errors) - this loses key information but should simplify the problem while preserving the basic relationship

log_df.dropna(inplace=True)

In [6]:
# Feature transformations

# Convert resp_time and bytes_setn into float

log_df.resp_time = log_df.resp_time.astype("float")
log_df.bytes_sent = log_df.bytes_sent.astype("float")

# Convert datetime string into Pandas Datetime

log_df.datetime = pd.to_datetime(log_df.datetime,
                                    format="%Y-%m-%d %H:%M:%S")

# Create categorical variables

log_df = pd.get_dummies(log_df, columns = ['resp_code'])
log_df = pd.get_dummies(log_df, columns = ['url'])

# Create column lists

url_features = [col for col in log_df.columns if "url" in col]
app_cols = [col for col in log_df.columns if "app_" in col]
resp_codes = [col for col in log_df.columns if "resp_code" in col]

print(log_df.shape)

(4250382, 141)


In [ ]:
# Create sequence prediction outcome variable -> next 250 requests

for url in url_features:
     log_df["CS_" + url] = log_df[url].rolling(250).sum()

cumsum_cols = [col for col in log_df.columns if "CS_" in col]

# Create rolling average request response time (over the next 250 requests)

log_df["av_rolling_resp_time_250"] = log_df["resp_time"].rolling(250).sum() / 250

# Delete NaN rows

log_df.dropna(inplace=True)

# Shift request_mix and response_time variables 250 places down

log_df[cumsum_cols] = log_df[cumsum_cols].shift(-250)
log_df["av_rolling_resp_time_250"] = log_df["av_rolling_resp_time_250"].shift(-250)
#log_df["error_rate"] = log_df["error_rate"].shift(-250)

# Delete NaN rows again

log_df.dropna(inplace=True)

# Scale key cols between 0 and 1

log_df[cumsum_cols] = MinMaxScaler().fit_transform(log_df[cumsum_cols])
log_df["av_rolling_resp_time_250"] = MinMaxScaler().fit_transform(log_df[["av_rolling_resp_time_250"]])
log_df[app_cols] = MinMaxScaler().fit_transform(log_df[app_cols])

In [ ]:
columns_to_drop = url_features + resp_codes + ["resp_time", "bytes_sent", "datetime"]

In [ ]:
log_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
log_df.to_parquet(path + 'resp_df.parquet')